In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [2]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')

In [3]:
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_train.drop_duplicates(inplace=True)

In [6]:
dataset = Dataset()

In [7]:
train_data, test_data = train_test_split(df_train, random_state=32, shuffle=True)

In [8]:
train_data.userid.max()

127495

In [9]:
dataset.fit(train_data.userid.unique(), train_data.itemid.unique())

In [10]:
dataset.model_dimensions()

(126552, 41137)

In [11]:
(interactions, weights) = dataset.build_interactions(
    [(x[11], x[12], x[13]) for x in train_data.values])

In [13]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [27]:
user_id_map[0]

27804

In [26]:
df_train.head()

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,userid,itemid,rating
0,5.0,True,"10 4, 2016",B01CPNIEQG,Heather,These are my FAVORITE spices in my collection....,Must Add to your Spice kitchen!,1475539200,NaN,NaN,NaN,102179,37138,1.0
1,5.0,True,"03 1, 2016",B006F63M8U,Linda Odom,Add A package to my Coffee and it makes a good...,Milk Chocolate Swiis MIss Hot Cocoa Mix,1456790400,NaN,{'Size:': ' 60-0.73 oz Envelopes'},NaN,3625,17322,1.0
2,5.0,True,"06 26, 2016",B00112O8NG,DesertBlossom,"I love the Torani syrups, but not the prices o...","Love these sugar free syrups, but didn't love ...",1466899200,28,NaN,NaN,39495,5600,1.0
3,5.0,True,"09 5, 2013",B00MRAQUM8,Connie L. Reaves,Very fragrant. The price can't be beat. I do l...,Nice !,1378339200,NaN,NaN,NaN,72854,30249,1.0
4,5.0,True,"09 14, 2016",B000R7RN0G,Mike P.,These are my favorite jalapeno chips so far. T...,These are my favorite jalapeno chips so far,1473811200,NaN,{'Flavor:': ' Jalapeo'},NaN,74859,4349,1.0


In [24]:
user_id_map[3625]

28927

In [30]:
model = LightFM(loss='warp')

In [31]:
model.fit(interactions, # spase matrix representing whether user u and item i interacted
      sample_weight= weights, # spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
      epochs=10)

In [33]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model,
                      interactions
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

Hybrid training set AUC: 0.90070546


In [39]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [40]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [13]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)

NameError: name 'model' is not defined

In [42]:
roc_auc_score(test_data.rating,preds)

0.7244333171669821

In [44]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values)

In [45]:
len(preds_sub)

285965

In [49]:
preds.min(), preds.max()

(-35.288516998291016, 38.3861083984375)

In [46]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [47]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [48]:
df_sub['rating']= normalized_preds

In [49]:
df_sub.to_csv('submission_log.csv', index=False)